## QHyper usecase

### Create instance of problem

In [10]:
import sys
sys.path.append('..')
from QHyper.problems.knapsack import KnapsackProblem
from QHyper.problems.tsp import TSPProblem

PROBLEM_TYPE = 'knapsack' # 'tsp'


# Each problem needs different parameters, because it depends on the number
# of variables and constraints
# Number of hyper_optimizer_bounds depends on the number of constraints,
# because each constraint requires one weights, and objective function also
# requires one
 
if PROBLEM_TYPE == 'knapsack':
    # Create knapsack with 3 items: 2 with weights 1 and value 2 - (1, 2),
    # and one with weight 1 and value 1 - (1, 1)
    problem = KnapsackProblem(max_weight=2, items=[(1, 2), (1, 2),(1, 1)])
    print(problem.knapsack.items)
    
    params_cofing = {
        'angles': [[0.5]*5, [1]*5],
        'hyper_args': [1, 2.5, 2.5],
    }
    hyper_optimizer_bounds = 3*[(1, 10)]

elif PROBLEM_TYPE == 'tsp':
    # Create Traveling Salesmam Problem with 3 cities
    problem = TSPProblem(
        number_of_cities=3,
    )
    params_cofing = {
        'angles': [[0.5]*5, [1]*5],
        'hyper_args': [1, 2, 2, 2, 2],
    }
    hyper_optimizer_bounds = 5*[(1, 10)]

[Item(weight=1, value=2), Item(weight=1, value=2), Item(weight=1, value=1)]


In [11]:
print(f"Variables used to describe objective function"
      f"and constraints: {problem.variables}")
print(f"Objective function: {problem.objective_function}")
print("Constraints:")
for constraint in problem.constraints:
    print(f"    {constraint}")

Variables used to describe objective functionand constraints: (x0, x1, x2, x3, x4)
Objective function: {('x0',): -2, ('x1',): -2, ('x2',): -1}
Constraints:
    {('x3',): -1, ('x4',): -1, (): 1}
    {('x0',): -1, ('x1',): -1, ('x2',): -1, ('x3',): 1, ('x4',): 2}


### Use VQA to solve knapsack problem

In [12]:
from QHyper.solvers.vqa.base import VQA
from QHyper.solvers import Solver

In [13]:
# Simple quantum circuit without optimzers will be used to test the results
# WF-QAOA is choosen becasue this PQC has most suitable evaluation function

tester_config = {
    'pqc': {
        'type': 'wfqaoa',
        'layers': 5,
    }
}

tester = VQA(problem, config=tester_config)

In [14]:
# Create a VQA instance with HQAOA as PQC and scipy optimizer
# This can be done in two various way
# 1. Providing dict with config (usefull to save experiment confing in e.g JSON)

solver_config = {
    "solver": {
        "type": "vqa",
        "args": {
            "config": {
                "optimizer": {
                    "type": "scipy",
                    "maxfun": 200,
                },
                "pqc": {
                    "type": "wfqaoa",
                    "layers": 5,
                }
            }
        }
    }
}

vqa = Solver.from_config(problem, solver_config)

# 2. Providing actual isntance of each class like VQA and Optimizer
from QHyper.solvers.vqa.pqc.h_qaoa import HQAOA
from QHyper.optimizers import ScipyOptimizer

vqa = VQA(problem, HQAOA(layers=5), ScipyOptimizer(maxfun=200))

In [15]:
# Run VQA with provided initial parameters
best_params = vqa.solve(params_cofing)
print(f"Best params: {best_params}")

Best params: {'angles': array([2.96486576e+00, 1.77950795e-01, 9.92993733e-01, 1.60313264e+00,
       4.03203090e+00, 1.38062854e+00, 4.93543621e-01, 8.15771852e-01,
       7.76871154e-04, 2.22518760e+00]), 'hyper_args': array([1.28432144, 2.92590948, 3.8769591 ])}


In [16]:
# Here created ealier tester will be used to evaluate results

best_results = tester.evaluate(best_params,print_results=True)
print(f"Best results: {best_results}")
print(f"Params used for optimizer:\n{best_params['angles']},\n"
      f"and params used for hyper optimizer: {best_params['hyper_args']}")

11001, 0.210, -4
00110, 0.193, -1
00001, 0.093, 0
00010, 0.068, 0
00101, 0.034, 0
10011, 0.033, 0
01011, 0.033, 0
01110, 0.033, 0
Best results: -1.1818289331882763
Params used for optimizer:
[2.96486576e+00 1.77950795e-01 9.92993733e-01 1.60313264e+00
 4.03203090e+00 1.38062854e+00 4.93543621e-01 8.15771852e-01
 7.76871154e-04 2.22518760e+00],
and params used for hyper optimizer: [1.28432144 2.92590948 3.8769591 ]


#### Using hyper optimizers

In [21]:
# Additionally other optimizer can be used to tune some parameters, in below
# example, Random optimzier will change weights (hyper_args) and choose ones
# that gives the best results after runnign 200 iteration of scipy minimizer

solver_config = {
    "solver": {
        "type": "vqa",
        "args": {
            "config": {
                "optimizer": {
                    "type": "scipy",
                    "maxfun": 200,
                },
                "pqc": {
                    "type": "wfqaoa",
                    "layers": 5,
                }
            }
        }
    },
    "hyper_optimizer": {
        "type": "random",    
        "processes": 1,
        "number_of_samples": 1,
        "bounds": hyper_optimizer_bounds
    }
}
vqa = Solver.from_config(problem, solver_config)

best_params_h = vqa.solve(params_cofing)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:22<00:00, 22.02s/it]


In [23]:
# Again, final results will be evaluated with tester

best_results = tester.evaluate(best_params_h, print_results=True)
print(f"Best results: {best_results}")
print(f"Params used for optimizer:\n{best_params_h['angles']},\n"
      f"and params used for hyper optimizer: {best_params_h['hyper_args']}")

00011, 0.145, 0
00001, 0.118, 0
00010, 0.087, 0
00111, 0.065, 0
11101, 0.056, 0
00110, 0.043, -1
01110, 0.040, 0
10110, 0.040, 0
Best results: -0.29767841104156945
Params used for optimizer:
[[1.64232476 0.08750321 0.         0.12856144 2.49365479]
 [0.93904305 0.6230146  1.10215501 0.41513281 1.45092844]],
and params used for hyper optimizer: [3.12174438 5.46520519 5.63104031]
